In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import requests, re, json ,pprint
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import urllib.request
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
from urllib.parse import urlparse
from time import sleep
import datetime
import pytz # new import
new_timezone = pytz.timezone("US/Eastern")
import spacy
import en_core_web_sm
import en_core_web_lg
import en_vectors_web_lg
import en_core_web_md
import csv
from dateparser.search import search_dates
import arrow
import datetime
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

In [2]:
driver_path='./chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')   
driver = webdriver.Chrome(driver_path,options=options)
# driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

In [3]:
def reload_test_site():
    my_url = 'https://www.youtube.com/watch?v=A1tqTQkC36A&amp;lc=Ugw0nq07e-G5JyUsl094AaABAg'
    driver.get(my_url)
    sleep(1)

In [4]:
def get_driver(driver,my_url):
#     my_url = 'https://www.youtube.com/watch?v=A1tqTQkC36A&amp;lc=Ugw0nq07e-G5JyUsl094AaABAg'
    driver.get(my_url)
    return driver
#     sleep(1)

## Define User ID

In [5]:
def extract_transcript(driver):
    sleep(2)
    driver.implicitly_wait(10)
    test = driver.find_element_by_xpath("//button[@aria-label='More actions']");
    test.click()
    driver.implicitly_wait(10)
    test_out = driver.find_element_by_xpath('//yt-formatted-string[text() ="Open transcript"]')
    test_out.click()
    driver.implicitly_wait(10)
    test_out = driver.find_element_by_xpath('//div[@id="body"]/ytd-transcript-body-renderer')
    test_body = test_out.text.encode('utf-8')    
    
    return test_body.decode('utf-8','ignore')

## Extract from List of Videos

In [ ]:
#define filename
csv_input = 'batch40_stefanmolyneux.csv'
csv_output = 'transcript_' + csv_input

In [24]:
#get file from cloud, skip if not necessary
!gsutil cp gs://ytscrape/$csv_input ./

Copying gs://ytscrape/batch19_prageru.csv...
/ [1 files][  4.2 GiB/  4.2 GiB]   38.9 MiB/s                                   
Operation completed over 1 objects/4.2 GiB.                                      


In [ ]:
#lazy load the whole file to dataframe
#wont work for large files due to memory constraint
# csv_input = 'batch19_prageru.csv'
# csv_output = 'transcript_' + csv_input
# filenames = pd.read_csv(csv_input)
# filenames.video_url.head()
# vid_id_df =filenames.video_url.unique()
# del filenames
# !rm batch19_prageru.csv 

In [14]:
#parse the video_url from big file

!echo video_url>temp
!sed -n '/^\"/p' $csv_input | sed -e 's/.*http/http/g' | sed '/^\"/d' | sed -e 's/\",.*//g' >>temp
filenames = pd.read_csv('temp')
vid_id_df =filenames.video_url.unique()
del filenames
display(len(vid_id_df))

352

In [ ]:
driver_path='./chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')   
driver = webdriver.Chrome(driver_path,options=options)
# driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

with open(csv_output, 'w', newline='') as csvfile:
    fieldnames = ['url', 'transcript']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames,
            delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for urls in vid_id_df:
        try:
            driver.get(urls)
            text = extract_transcript(driver)
            x = {"url":urls,"transcript":text}
            writer.writerow(x)
        except:
            pass
print('done')

In [ ]:
!gsutil cp $csv_output gs://yttranscript/